In [169]:
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

import scipy
import nrrd
import argparse
import random
import multiprocessing

from collections import OrderedDict 

out_path = "./dataset"

In [170]:
labels = pd.read_csv("./raw/data.csv")
labels.sort_values(['id', 'scan_date'], inplace=True)
labels

,id,scan_date,path,birth,tbv
135,1,01/02/2018,./raw/01/01_X_20180201.nrrd,12/01/2018,181.52
138,1,01/03/2018,./raw/01/01_X_20180301.nrrd,12/01/2018,255.22
139,1,08/02/2018,./raw/01/01_X_20180208.nrrd,12/01/2018,192.31
131,1,08/03/2018,./raw/01/01_X_20180308.nrrd,12/01/2018,261.03
134,1,16/02/2018,./raw/01/01_X_20180216.nrrd,12/01/2018,218.43
...,...,...,...,...,...
118,87,15/07/2019,./raw/87/87_X_20190715.nrrd,16/06/2019,132.57
120,87,21/06/2019,./raw/87/87_X_20190621.nrrd,16/06/2019,99.47
117,87,22/07/2019,./raw/87/87_X_20190722.nrrd,16/06/2019,149.17
115,87,26/06/2019,./raw/87/87_X_20190626.nrrd,16/06/2019,99.51


In [173]:
target_size = 96

for _, row in labels.iterrows():

    # Load nrrd
    new_path = os.path.join(out_path, f"{row['id']}_{row['scan_date'].replace('/', '-')}.nrrd")
    
    data, headers = nrrd.read(row['path'])

    old_sizes = headers['sizes']
    old_spacings = headers['spacings']

    resize_factors = [target_size/old_sizes[0], target_size/old_sizes[1], target_size/old_sizes[2]]

    data = scipy.ndimage.zoom(data, resize_factors)
    new_sizes = data.shape
    new_spacing = [old_spacings[0] * old_sizes[0] / new_sizes[0], old_spacings[1] * old_sizes[1] / new_sizes[1], old_spacings[2] * old_sizes[2] / new_sizes[2]]

    total_volume = np.prod(old_sizes) * np.prod(old_spacings)
    new_tbv_voxels = round(row['tbv'] * np.prod(new_sizes) / total_volume, 2)

    age = (datetime.datetime.strptime(row['scan_date'], '%d/%m/%Y') - datetime.datetime.strptime(row['birth'], '%d/%m/%Y')).days

    new_headers = OrderedDict({
        'type': 'unsigned char',
        'dimension': 3,
        'sizes': new_sizes,
        'spacings': new_spacing,
        'kinds': ['space', 'space', 'space'],
        'labels': ['x', 'y', 'z'],
        'encoding': 'raw',
        'units': ['cm', 'cm', 'cm'],
        'tbv_n_voxels': new_tbv_voxels,
        'tbv': row['tbv'],
        'age_days': age
    })

    nrrd.write(new_path, data, new_headers)


12/01/2018
01/02/2018
20
12/01/2018
01/03/2018
48
12/01/2018
08/02/2018
27
12/01/2018
08/03/2018
55
12/01/2018
16/02/2018
35
12/01/2018
19/01/2018
7
12/01/2018
22/01/2018
10
12/01/2018
22/02/2018
41
12/01/2018
26/01/2018
14
29/01/2018
02/02/2018
4
29/01/2018
05/02/2018
7
29/01/2018
14/02/2018
16
29/01/2018
01/03/2018
31


KeyboardInterrupt: 